In [1]:
import lovely_tensors as lt

lt.monkey_patch()
from ocpmodels.models.rhescnv8clean.tests.utils import test_context

rhctx = test_context("/workspaces/ocp/configs/s2ef/2M/escn/rhescn-ckpt.yml").__enter__()
escnctx = test_context("/workspaces/ocp/configs/s2ef/2M/escn/escn-ckpt.yml").__enter__()

2024-04-07 02:16:26 (INFO): Project root: /workspaces/ocp


2024-04-07 02:16:27 (WARNING): Detected old config, converting to new format. Consider updating to avoid potential incompatibilities.


2024-04-07 02:16:27 (INFO): amp: true
cmd:
  checkpoint_dir: ./checkpoints/2024-04-07-02-16-32
  commit: 96a7c9b
  identifier: ''
  logs_dir: ./logs/wandb/2024-04-07-02-16-32
  print_every: 10
  results_dir: ./results/2024-04-07-02-16-32
  seed: 0
  timestamp_id: 2024-04-07-02-16-32
dataset:
  format: trajectory_lmdb
  grad_target_mean: 0.0
  grad_target_std: 2.887317180633545
  key_mapping:
    force: forces
    y: energy
  normalize_labels: true
  src: /mnt/datasets/s2ef/2M/train/
  target_mean: -0.7554450631141663
  target_std: 2.887317180633545
  transforms:
    normalizer:
      energy:
        mean: -0.7554450631141663
        stdev: 2.887317180633545
      forces:
        mean: 0.0
        stdev: 2.887317180633545
eval_metrics:
  metrics:
    energy:
    - mae
    forces:
    - forcesx_mae
    - forcesy_mae
    - forcesz_mae
    - mae
    - cosine_similarity
    - magnitude_error
    misc:
    - energy_forces_within_threshold
gpus: 1
logger: wandb
loss_fns:
- energy:
    coeffic

2024-04-07 02:16:29 (WARNING): Detected old config, converting to new format. Consider updating to avoid potential incompatibilities.


2024-04-07 02:16:29 (INFO): amp: true
cmd:
  checkpoint_dir: ./checkpoints/2024-04-07-02-16-32
  commit: 96a7c9b
  identifier: ''
  logs_dir: ./logs/wandb/2024-04-07-02-16-32
  print_every: 10
  results_dir: ./results/2024-04-07-02-16-32
  seed: 0
  timestamp_id: 2024-04-07-02-16-32
dataset:
  format: trajectory_lmdb
  grad_target_mean: 0.0
  grad_target_std: 2.887317180633545
  key_mapping:
    force: forces
    y: energy
  normalize_labels: true
  src: /mnt/datasets/s2ef/2M/train/
  target_mean: -0.7554450631141663
  target_std: 2.887317180633545
  transforms:
    normalizer:
      energy:
        mean: -0.7554450631141663
        stdev: 2.887317180633545
      forces:
        mean: 0.0
        stdev: 2.887317180633545
eval_metrics:
  metrics:
    energy:
    - mae
    forces:
    - forcesx_mae
    - forcesy_mae
    - forcesz_mae
    - mae
    - cosine_similarity
    - magnitude_error
    misc:
    - energy_forces_within_threshold
gpus: 1
logger: wandb
loss_fns:
- energy:
    coeffic

In [8]:
rhgrid = rhctx.trainer.model.layer_blocks[0].conv_rh_grid
escngrid = escnctx.trainer.model.SO3_grid[escnctx.trainer.model.lmax_list[0]][
    escnctx.trainer.model.lmax_list[0]
]
escngrid._initialize(escnctx.trainer.device)

print(rhgrid.res_beta, rhgrid.res_alpha)
print(escngrid.lat_resolution, escngrid.long_resolution)

print(rhgrid, escngrid)
print(rhgrid.to_grid_sh_tri, escngrid.to_grid_mat)

10 11
10 11
Rhomboidal_SO3_Grid_Optimized() SO3_Grid()
tensor[110, 81] n=8910 (35Kb) x∈[-0.872, 0.872] μ=0.009 σ=0.282 cuda:0 tensor[10, 11, 25] n=2750 (11Kb) x∈[-0.692, 0.745] μ=0.022 σ=0.281 cuda:0


In [3]:
import numpy as np
import rich
import torch
from bidict import bidict
from einops import rearrange

LMAX = 4
MMAX = 2
RH_MMAX = 4


def trimask(x: np.ndarray, lmax: int = LMAX):
    return x[:, : (lmax + 1) ** 2]


def _escnindices(lmax: int, mmax: int):
    coeffs = bidict[tuple[int, int], int]()
    for l in range(lmax + 1):
        for m in range(-l, l + 1):
            if abs(m) > mmax:
                continue

            coeffs[(l, m)] = len(coeffs)
    return coeffs


def trimask_mmax(
    x: np.ndarray,
    lmax: int = LMAX,
    mmax: int = MMAX,
):
    indices: list[int] = []
    i = 0
    for l in range(lmax + 1):
        for m in range(-l, l + 1):
            if abs(m) > mmax:
                i += 1
                continue

            indices.append(i)
            i += 1

    return x[:, indices]


def rhmask(
    x: torch.Tensor,
    lmax: int = LMAX,
    mmax: int = MMAX,
    rh_mmax: int = RH_MMAX,
):
    if x.ndim == 3:
        x = rearrange(
            x,
            "E (m two_sign l) C -> E m two_sign l C",
            m=rh_mmax + 1,
            two_sign=2,
        )

    idx = _escnindices(lmax, mmax)
    x_masked = x.new_zeros((x.shape[0], x.shape[1], len(idx)))
    for (l, m), i in idx.items():
        signidx = int(np.signbit(m).item())
        x_masked[:, :, i] = x[:, :, abs(m), signidx, l - abs(m)]
    return x_masked


to_grid_sh_rh = rearrange(
    rhgrid.to_grid_sh_rh,
    "(res_beta res_alpha) (m two l) -> res_beta res_alpha m two l",
    res_beta=10,
    two=2,
    m=5,
)
rich.print(
    {
        "rh": to_grid_sh_rh,
        "escn": escngrid.to_grid_mat,
    }
)
rich.print({"rh": rhmask(to_grid_sh_rh)})
torch.testing.assert_close(rhmask(to_grid_sh_rh), escngrid.to_grid_mat)

{
    'rh': tensor[10, 5, 5, 2, 5] n=2500 (9.8Kb) x∈[-0.770, 0.770] μ=0.017 σ=0.293 cuda:0,
    'escn': tensor[10, 5, 19] n=950 (3.7Kb) x∈[-0.692, 0.745] μ=0.029 σ=0.293 cuda:0
}

{'rh': tensor[10, 5, 19] n=950 (3.7Kb) x∈[-0.692, 0.745] μ=0.029 σ=0.293 cuda:0}

In [4]:
from ocpmodels.models.rhescnv8clean.so3 import _grid_precomputes_optimized

lmax = escnctx.trainer.model.lmax_list[0]

to_grid_sh_tri, from_grid_sh_tri = _grid_precomputes_optimized(
    4,
    escngrid.lat_resolution,
    escngrid.long_resolution,
    normalization="integral",
)
to_grid_sh_tri[:, :, :25]

tensor[10, 5, 25] n=1250 (4.9Kb) x∈[-0.692, 0.745] μ=0.022 σ=0.281